In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import data
data = pd.read_csv('../../Data/Fremont_Bridge_Bicycle_Counter.csv', index_col='Date', parse_dates=True)
data.columns = ['Total', 'East', 'West']

In [ ]:
data.sort_index(inplace=True)

In [ ]:
data_daily = data.resample('D').sum()

In [ ]:
# plot daily traffic, data currently hourly
data_daily['Total'].resample('D').sum().plot()
plt.ylabel('Daily bicycle traffic')
plt.show()

#### Goal: Predict daily traffic based on:
- day of the week
- year (upward trend? more ppl)
- holidays
- hours of daylight
- weather (temp, precipitation, wind speed, etc)

In [ ]:
weather_data = pd.read_csv('../../Data/Seattle_weather_102012_082023.csv', index_col='DATE', parse_dates=True)

In [ ]:
# add times features
data_daily['Day'] = data_daily.index.dayofweek
data_daily['Month'] = data_daily.index.month
data_daily['Year'] = data_daily.index.year
# 'covid column' 0 pre covid, 1 post covid (2020-03-01)
data_daily['Covid'] = np.where(data_daily.index >= '2020-03-01', 1, 0)

In [ ]:
# add holiday feature
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=data_daily.index.min(), end=data_daily.index.max())
data_daily['Holiday'] = data_daily.index.isin(holidays)

In [ ]:
data_daily.loc[data_daily['Holiday'] == 1]

In [ ]:
def get_hours_daylight(date):
    axis = np.radians(23.44) # tilt of earth axis
    latitude = np.radians(47.61) # seattle lat
    days = (date - pd.to_datetime('2000-12-31')).days

    m = (1 - np.tan(latitude) * np.tan(axis*np.cos(days*2*np.pi/365.25)))
    hours_of_daylight = 24 * np.degrees(np.arccos(1 - m)) / 180
    return hours_of_daylight

In [ ]:
data_daily['hours_of_daylight'] = data_daily.index.map(get_hours_daylight)

We will use weather features:

- PRCP: Precipitation
- TAVG: Average Temperature
- SNOW: Snowfall
- AWND: Average wind speed

In [ ]:
cols_to_keep = ['PRCP', 'TAVG', 'SNOW', 'AWND']
weather_data2 = weather_data[cols_to_keep]

In [ ]:
weather_data.TAVG.fillna((0.5*weather_data.TMAX + 0.5*weather_data.TMIN), inplace=True)
weather_data.TAVG.plot()

In [ ]:
# add weather data columns
data_daily['PRCP'] = weather_data2.PRCP
data_daily['TAVG'] = weather_data2.TAVG
data_daily['SNOW'] = weather_data2.SNOW
data_daily['AWND'] = weather_data2.AWND

In [ ]:
data_daily

In [ ]:
# feature matrix X, target vector y
X = data_daily.drop(['Total', 'East', 'West', 'Month'], axis=1)
y = data_daily['Total']

### Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# create pipeline
numeric_features = ['Year', 'hours_of_daylight', 'PRCP', 'TAVG', 'SNOW', 'AWND']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())])

categorical_features = ['Covid', 'Holiday', 'Day']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipe_model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('poly', PolynomialFeatures(degree=1)),
                        ('lin_reg', LinearRegression())])

pipe_model_all_data = Pipeline(steps=[('preprocessor', preprocessor),
                        ('poly', PolynomialFeatures(degree=1)),
                        ('lin_reg', LinearRegression())])

In [ ]:
pipe_model_all_data.fit(X, y)
data_daily['predicted'] = pipe_model_all_data.predict(X)

In [ ]:
# plot daily trips actual vs predicted
data_daily[['Total', 'predicted']].plot(alpha=0.5, figsize=(10, 6))
plt.ylabel('Daily bicycle traffic')
plt.show()

In [ ]:
data_daily.plot.scatter(x='Total', y='predicted', alpha=0.2)
# add regression line
x = np.linspace(*plt.xlim())
plt.plot(x, x, color='black')
plt.show()

In [ ]:
# monthly values
data_daily[['Total', 'predicted']].resample('M').sum().plot(alpha=0.9, figsize=(10, 6))

In [ ]:
# yearly values
data_daily[['Total', 'predicted']].resample('Y').sum().plot(alpha=0.9, figsize=(10, 6))

#### Coefficients

In [ ]:
pipe_model_all_data['preprocessor'].get_feature_names_out()

In [ ]:
feature_names = pipe_model_all_data['poly'].get_feature_names_out(input_features=pipe_model_all_data['preprocessor'].get_feature_names_out())

In [ ]:
coefs = pd.DataFrame({'coef': pipe_model_all_data['lin_reg'].coef_}, index=feature_names)

In [ ]:
coefs

In [ ]:
# plot feature importances
from sklearn.inspection import permutation_importance

result = permutation_importance(pipe_model_all_data, X, y, n_repeats=10, random_state=42, n_jobs=2)

sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(result.importances[sorted_idx].T, vert=False, labels=X.columns[sorted_idx])
ax.set_title("Permutation Importances")
fig.tight_layout()
plt.show()

### Regularization + polynomial features + hyperparameter tuning

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso

In [ ]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# create pipeline
pipe_model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('poly', PolynomialFeatures(degree=1)),
                        ('reg', Lasso())
                    ])

In [148]:
# define parameter grid
param_grid = {
    'poly__degree': [1, 2, 3],
    'reg__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# create grid search
grid = GridSearchCV(pipe_model, param_grid, cv=5, n_jobs=-1, verbose=1, scoring='neg_mean_squared_error') # scoring='neg_mean_absolute_error', we want to minimize the MSE, so negative MAE

# fit grid search
grid.fit(X_train, y_train)

# best parameters
grid.best_params_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.483e+06, tolerance: 3.904e+05
  model = cd_fast.enet_coordinate_descent(
/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.615e+06, tolerance: 3.959e+05
  model = cd_fast.enet_coordinate_descent(
/home/jake/miniconda3/envs/sportsenv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

KeyboardInterrupt: 